#Request

In [65]:
import requests

csv_url = "https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv?departamento_nom=AMAZONAS&accessType=DOWNLOAD&api_foundry=true"
#csv_url = "./content/sample_data/sample_downloaded.csv"
root_path = "/content/drive/MyDrive/Salisbury/data/covid_downloaded.csv"
def downloadDataCsv(csv_url,root_path):
  req = requests.get(csv_url)
  url_content = req.content
  try:
      csv_file = open(root_path, 'wb')

      csv_file.write(url_content)

      csv_file.close()
      print("I finished")

  except:
      print("something was wrong")

#downloadDataCsv(csv_url,root_path)

#Model

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import math

import os
#!pip install sodapy -q
#!pip install socrata-py -q

#from sodapy import Socrata
#from socrata.authorization import Authorization


In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [53]:
def requestColombianData(limit = 1000):
  client = Socrata(
                  "www.datos.gov.co",
                  "idLwSviPi5dGMTd7GfIfW3CLU",
                  #username="sebastian_romerola@fet.edu.co",
                  #password="Sebastian#2018",
                  timeout=300
                  )
  results = client.get("gt2j-8ykr",
                      #  ciudad_municipio_nom="Neiva",
                      #  departamento_nom="HUILA",
                      limit=limit
                      )
  
  return pd.DataFrame.from_records(results)


def getCsvDataSet(path='/content/drive/MyDrive/Salisbury/data/covid_downloaded.csv'):
  return pd.read_csv(r''+path)

In [72]:
df = getCsvDataSet()
df.head(10)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (14,17,19,20,22) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,fecha reporte web,ID de caso,Fecha de notificación,Código DIVIPOLA departamento,Nombre departamento,Código DIVIPOLA municipio,Nombre municipio,Edad,Unidad de medida de edad,Sexo,Tipo de contagio,Ubicación del caso,Estado,Código ISO del país,Nombre del país,Recuperado,Fecha de inicio de síntomas,Fecha de muerte,Fecha de diagnóstico,Fecha de recuperación,Tipo de recuperación,Pertenencia étnica,Nombre del grupo étnico
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380.0,ITALIA,Recuperado,27/2/2020 0:00:00,NaN,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6.0,NaN
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,4/3/2020 0:00:00,NaN,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5.0,NaN
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,29/2/2020 0:00:00,NaN,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6.0,NaN
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,8/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN
5,11/3/2020 0:00:00,6,10/3/2020 0:00:00,5,ANTIOQUIA,5360,ITAGUI,27,1,F,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
6,11/3/2020 0:00:00,7,8/3/2020 0:00:00,13001,CARTAGENA,13001,CARTAGENA,85,1,F,Importado,Casa,Leve,840.0,ESTADOS UNIDOS DE AMÉRICA,Recuperado,2/3/2020 0:00:00,NaN,11/3/2020 0:00:00,17/3/2020 0:00:00,PCR,6.0,NaN
7,11/3/2020 0:00:00,8,9/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,22,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6.0,NaN
8,11/3/2020 0:00:00,9,8/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,28,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,7/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN
9,12/3/2020 0:00:00,10,12/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,36,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,6/3/2020 0:00:00,NaN,12/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6.0,NaN


In [ ]:
#dataset from API
#df = None
#df = requestColombianData(460)

In [73]:
df.head(20)

,fecha reporte web,ID de caso,Fecha de notificación,Código DIVIPOLA departamento,Nombre departamento,Código DIVIPOLA municipio,Nombre municipio,Edad,Unidad de medida de edad,Sexo,Tipo de contagio,Ubicación del caso,Estado,Código ISO del país,Nombre del país,Recuperado,Fecha de inicio de síntomas,Fecha de muerte,Fecha de diagnóstico,Fecha de recuperación,Tipo de recuperación,Pertenencia étnica,Nombre del grupo étnico
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380.0,ITALIA,Recuperado,27/2/2020 0:00:00,NaN,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6.0,NaN
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,4/3/2020 0:00:00,NaN,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5.0,NaN
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,29/2/2020 0:00:00,NaN,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6.0,NaN
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,8/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN
5,11/3/2020 0:00:00,6,10/3/2020 0:00:00,5,ANTIOQUIA,5360,ITAGUI,27,1,F,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
6,11/3/2020 0:00:00,7,8/3/2020 0:00:00,13001,CARTAGENA,13001,CARTAGENA,85,1,F,Importado,Casa,Leve,840.0,ESTADOS UNIDOS DE AMÉRICA,Recuperado,2/3/2020 0:00:00,NaN,11/3/2020 0:00:00,17/3/2020 0:00:00,PCR,6.0,NaN
7,11/3/2020 0:00:00,8,9/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,22,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6.0,NaN
8,11/3/2020 0:00:00,9,8/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,28,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,7/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN
9,12/3/2020 0:00:00,10,12/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,36,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,6/3/2020 0:00:00,NaN,12/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6.0,NaN


In [74]:
print(df.columns)
df.columns = [
  'web_date_report',
  'case_id',
  'notification_date',
  'davipola_departament_code',
  'department_name',
  'davipola_municipality_code',
  'municipality_name',
  'age',
  'age_measurement_unit',
  'sex',
  'contagion_type',
  'case_location',
  'state',
  'iso_country_code',
  'country_name',
  'recuperated', #spell correctly
  'beginning_symptoms_date',#!/usr/bin/env python
  'date_of_death',
  'date_of_diagnosis',
  'date_of_recovery',
  'recovery_type',
  'ethnicity',
  'ethnic_group_name',
]


Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
       'Código DIVIPOLA departamento', 'Nombre departamento',
       'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad',
       'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
       'Ubicación del caso', 'Estado', 'Código ISO del país',
       'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas',
       'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación',
       'Tipo de recuperación', 'Pertenencia étnica',
       'Nombre del grupo étnico'],
      dtype='object')


In [57]:
print(df.isnull().sum())

web_date_report                     0
case_id                             0
notification_date                   0
davipola_departament_code           0
department_name                     0
davipola_municipality_code          0
municipality_name                   0
age                                 0
age_measurement_unit                0
sex                                 0
contagion_type                      0
case_location                   15212
state                           15212
iso_country_code              4527553
country_name                  4527545
recuperated                     12806
beginning_symptoms_date        516910
date_of_diagnosis             4402058
date_of_recovery                 4106
recovery_type                  254145
ethnicity                      254145
ethnic_group_name               58988
date_of_death                 4467698
dtype: int64


In [75]:
df.astype = {  
  "web_date_report": np.str,
  "case_id": np.int, #ordinal
  "notification_date": np.str, 
  "davipola_departament_code": np.int, #categorical
  "department_name": np.str, #categorical
  "davipola_municipality_code": np.str, #categorical
  "municipality_name": np.str, #categorical
  "age": np.int, 
  "age_measurement_unit": np.int,
  "sex": np.str, #nominal
  "contagion_type": np.str, #categorical
  "case_location": np.str, #categorical
  "state": np.str, #ordinal
  "iso_country_code": np.int, #categorical
  "country_name": np.str, #categorical
  "recuperated": np.str, ##categorical
  "beginning_symptoms_date": np.str, #!/usr/bin/env python
  "date_of_diagnosis": np.str,
  "date_of_recovery": np.str,
  "recovery_type": np.str, #categorica
  "ethnicity": np.int, #nominal
  "ethnic_group_name": np.str, #nominal
  "date_of_death": np.str,
}

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
np.shape(df)

In [ ]:
df.columns

In [ ]:
df = df.fillna(0)

In [ ]:
df_string = df.loc[:, [
  "web_date_report",
  "notification_date",
  "department_name",
  "municipality_name",
  "sex",
  "contagion_type",
  "case_location",
  "state",
  "country_name",
  "recuperated",
  "beginning_symptoms_date",
  "date_of_diagnosis",
  "date_of_recovery",
  "recovery_type",
  "ethnic_group_name",
  "date_of_death",
]].copy()


In [ ]:
df_string

uppercase to each str row

In [ ]:
df.select_dtypes(include='object')

In [ ]:
df.loc[:, df.dtypes == object]

df.select_dtypes(include='object')

In [ ]:
df.describe()

In [ ]:
df.info()

Se quitan los nulos y se reemplazan por 0

verifying nulls


```
print(df.isnull().values.any())
```
[source](https://towardsdatascience.com/a-complete-guide-to-principal-component-analysis-pca-in-machine-learning-664f34fc3e5a)

After use this function, the result of the code is false.


In [ ]:
print(df.isnull().values.any())

In [ ]:
df.date_of_death


In [ ]:
df.date_of_recovery

In [ ]:
df.info()

Similarity mesurement (radio basics funciotn)
1.  
*   ML / rbf -| knn graph / 
*   train to know about who would most highly survive
*   KNN graph (find the most near neibough
2.  
*   svm (support vector machine)





Adjency matrix. to build a graph after apply rbf and knn aproach?



In [ ]:
"""
model = tf.keras.Sequential([
  tf.layers.Dense(input_shape=(28,28,3))),
  tf.layers.Dense(100, activation='relu'),
  tf.layers.Dense(100, activation='relu'),
  tf.layers.Dense(10, activation='softmax')
])
"""

In [42]:
str_ = df.date_of_death[0]
head, sep, tail = str_.partition(' ')
head

'17/7/2020'

In [77]:
df.head(1700)

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380.0,ITALIA,Recuperado,27/2/2020 0:00:00,NaN,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6.0,NaN
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,4/3/2020 0:00:00,NaN,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5.0,NaN
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,29/2/2020 0:00:00,NaN,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6.0,NaN
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,8/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,7/4/2020 0:00:00,1696,4/4/2020 0:00:00,76,VALLE,76001,CALI,58,1,F,Comunitaria,Casa,Leve,NaN,NaN,Recuperado,1/4/2020 0:00:00,NaN,7/4/2020 0:00:00,21/4/2020 0:00:00,PCR,6.0,NaN
1696,7/4/2020 0:00:00,1697,6/4/2020 0:00:00,76,VALLE,76001,CALI,67,1,F,Comunitaria,Casa,Leve,NaN,NaN,Recuperado,1/4/2020 0:00:00,NaN,7/4/2020 0:00:00,26/4/2020 0:00:00,PCR,6.0,NaN
1697,7/4/2020 0:00:00,1698,4/4/2020 0:00:00,76,VALLE,76001,CALI,64,1,M,Relacionado,Fallecido,Fallecido,NaN,NaN,Fallecido,23/3/2020 0:00:00,31/5/2020 0:00:00,7/4/2020 0:00:00,NaN,NaN,6.0,NaN
1698,7/4/2020 0:00:00,1699,27/3/2020 0:00:00,76,VALLE,76001,CALI,36,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,12/3/2020 0:00:00,NaN,7/4/2020 0:00:00,16/6/2020 0:00:00,PCR,6.0,NaN


In [79]:
df["beginning_symptoms_date"] = pd.to_datetime(df["beginning_symptoms_date"],format='%d/%m/%Y %H:%M:%S')


In [83]:
df["beginning_symptoms_date"][0] < df["beginning_symptoms_date"][1]

True

In [66]:
df["beginning_symptoms_date"] = pd.to_datetime(df["beginning_symptoms_date"],format='%d/%m/%Y %H:%M:%S')
df["date_of_death"] = df.date_of_death.str.replace(' 0:00:00', '').str.replace('/', '')
df["web_date_report"] = df.web_date_report.str.replace(' 0:00:00', '').str.replace('/', '')
df["date_of_recovery"] = df.date_of_recovery.str.replace(' 0:00:00', '').str.replace('/', '')
df["date_of_diagnosis"] = df.date_of_diagnosis.str.replace(' 0:00:00', '').str.replace('/', '')
df["notification_date"] = df.notification_date.str.replace(' 0:00:00', '').str.replace('/', '')

In [37]:
for j in range(len(df.index)):
  if isinstance(df.beginning_symptoms_date[j], str):
    str_ =  df.beginning_symptoms_date[j]
    head = str_.split()
    df.beginning_symptoms_date[j] = int((head[0]).replace("/", ""))

    # print(df.beginning_symptoms_date[j])
  

ValueError: ignored

In [40]:

for i in df.columns:
  if i.find("date") != -1:
    print(df[i][0])
#    error = list()
#    a = list()
    for j in range(len(df.index)):
      str_ = df[i][j]
      head, sep, tail = str_.partition(' ')
      df[i][j] = int((head).replace("/", ""))
      print(df[i][j])
      #if j = null
#       try:
#         if j != 0:
#           a.append(datetime.strptime(str(j), '%d/%m/%Y %H:%M:%S').timestamp())
#         else:
#           a.append(0)
#         df[i] = all
#       except:
#         #print("the value "+str(j)+" was not correctly converted")
#         a.append(0)
# #        error.append([j,i])


6/3/2020 0:00:00
632020
932020
932020
1132020
1132020
1132020


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Streaming output truncated to the last 5000 lines.
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
1772020
17720

KeyboardInterrupt: ignored

In [9]:
for i in df.select_dtypes(include='object'):
  df[i] = df[i].str.upper()
  #  print(df[i])
  

In [41]:
df.head()

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name,date_of_death
0,632020,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,IMPORTADO,CASA,LEVE,380.0,ITALIA,RECUPERADO,27/2/2020 0:00:00,NaN,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6.0,632020
1,932020,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,4/3/2020 0:00:00,NaN,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5.0,NaN
2,932020,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,29/2/2020 0:00:00,NaN,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6.0,NaN
3,1132020,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
4,1132020,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,8/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN


In [40]:
df.dtypes

web_date_report                object
case_id                         int64
notification_date              object
davipola_departament_code       int64
department_name                object
davipola_municipality_code      int64
municipality_name              object
age                             int64
age_measurement_unit            int64
sex                            object
contagion_type                 object
case_location                  object
state                          object
iso_country_code              float64
country_name                   object
recuperated                    object
beginning_symptoms_date        object
date_of_diagnosis              object
date_of_recovery               object
recovery_type                  object
ethnicity                      object
ethnic_group_name             float64
date_of_death                  object
dtype: object

1.   Delete departament name && municipality name 
2.   Binarize sex column
3.   Delete country name
4.   Normalize recovery type
5.   Write documentation


In [41]:
df.describe()

,case_id,davipola_departament_code,davipola_municipality_code,age,age_measurement_unit,iso_country_code,ethnic_group_name
count,4.530610e+06,4.530610e+06,4.530610e+06,4.530610e+06,4.530610e+06,4.530610e+06,4.530610e+06
mean,2.265345e+06,1.302291e+03,2.804815e+04,3.969038e+01,1.003440e+00,4.034152e-01,5.830405e+00
std,1.307875e+06,5.853340e+03,2.598089e+04,1.788452e+01,6.372697e-02,1.720835e+01,9.007966e-01
min,1.000000e+00,5.000000e+00,5.001000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.132693e+06,1.100000e+01,1.100100e+04,2.700000e+01,1.000000e+00,0.000000e+00,6.000000e+00
50%,2.265346e+06,1.700000e+01,1.100100e+04,3.700000e+01,1.000000e+00,0.000000e+00,6.000000e+00
75%,3.397998e+06,6.800000e+01,5.000100e+04,5.200000e+01,1.000000e+00,0.000000e+00,6.000000e+00
max,4.530650e+06,4.700100e+04,9.977300e+04,1.140000e+02,3.000000e+00,1.724000e+03,6.000000e+00


Analisis de componentes principales (APC)

quitar lo que se puede.



desviación estandar de  para analizar causas de muerte

*italicized text*.fit

se realiza conversión a todos los que tengan

In [ ]:
#////////////////////////////////////////////////////////#
#                                                        /
#      Binarize Sex column values to avoid errors        /
#                                                        /
#////////////////////////////////////////////////////////#
from sklearn.preprocessing import Binarizer



#/-////////////////////////////////////////////////////////#
#                                                         /
#              average of recovery time per genre         /
#                                                         /
#/////////////////////////////////////////////////////////#

#////////////////////////////////////////////////////////#
#                                                        /
#     Capitalize Sex column values to avoid errors       /
#                                                        /
#////////////////////////////////////////////////////////#




#////////////////////////////////////////////////////////#
#                                                        /
#  substract betweeen two dates and store it the answer  /
#                                                        /
#////////////////////////////////////////////////////////#

def get_duration_of_illness(x:any):
  time = 0
  z = list()
  count = 0
  for e in x.index:
    count += 1
    if x.date_of_recovery[e] != 0:
      x.loc[e, "time_to_recover"] = abs(x.date_of_recovery[e] - x.beginning_symptoms_date[e])
    elif x.date_of_death[e] != 0:
      x.loc[e, "time_to_recover"] = abs(x.date_of_death[e] - x.beginning_symptoms_date[e])
    elif x.date_of_death[e] == 0 and x.date_of_recovery[e] == 0:
      print("NAN")
      x.loc[e, "time_to_recover"] = 0
  
  print("The total amount of data is: "+ str(count))

  return x

      # x.duration_of_illness.append()
      
      # abs((d2 - d1).days)

#////////////////////////////////////////////////////////#
#                                                        /
# Get important Columns to get the main of recovery time /
#                                                        /
#////////////////////////////////////////////////////////#

mean = 0


#it is not necessary use another array, a better approach may be use df directly
array_duration_of_illness = df.loc[
                                   :,
                                   [
                                    "sex",
                                    "beginning_symptoms_date",
                                    "date_of_recovery",
                                    "date_of_death",
                                    "recuperated"
                                    ]
                                   ].copy()

                                   #  sex M/ 15 avg to get recover from beginning_symptoms_date. 
                                   #  sex F/ 14 avg.

                                   # Male / Ibague / how long time do they neeed to get recover?
                                   #cound .. avg ... 

                                   ## share tje ML research results -| graph clustering. 
array_dates_women = df[df["sex"] == 'F']
get_duration_of_illness(array_dates_women)

array_dates_men = df[df["sex"] == "M"]
get_duration_of_illness(array_dates_men)

#////           For alive people
# df.beginning_symptoms_date - df.date_of_recovery 


#////           For death people
# df.beginning_symptoms_date - df.date_of_death 

In [ ]:
df.isnull().sum()

Replacing


In [ ]:
#df.boxplot('age','contagion_type',rot = 0,figsize=(5,6))

In [ ]:
df.dtypes.value_counts()

#Important variable content


Posible ways to replace
[Pandas Documentation 
](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html)
```
df.replace({'A': {0: 100, 4: 400}})
Result:
        A  B  C
0  100  5  a
1    1  6  b
2    2  7  c
3    3  8  d
4  400  9  e
```



# Data Transformation
label or one hot enconding or replacing



In [ ]:
#   df.replace([{'sex': {'F': 1, 'M': -1}]})
df.replace(['F',
            'M'],
          [ '1',
            '-1'])

In [ ]:

#   Female = 0; Male = 1

#   Female = 0; Male = 1
'''
#contagion type
df.replace(['Comunitaria',
            'Relacionado',
            'En estudio',
            'Importado',],
           ['0',
            '1',
            '2',
            '3',])

df.replace(['casa',
            'Hospital',
            'Hospital UCI',
#   Its important because of the equivalent vallues in other columns,
#   so, in other columns also will be replaced by the same nuemric vallue
            'N/A',
            'Fallecido',
            ],
           ['0',
            '1',
            '2',
            '3',
            '4',])


df.replace(['leve',
            'Moderado',
            'Grave',
#   Its important because of the equivalent vallues in other columns,
#   so, in other columns also will be replaced by the same nuemric vallue
            'N/A',
            'Fallecido',
            ],
           ['0',
            '1',
            '2',
            '3',
            '4',])

'''

In [42]:
df.head(20)

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name,date_of_death
0,<built-in function all>,1,<built-in function all>,11,BOGOTA,11001,BOGOTA,19,1,F,IMPORTADO,CASA,LEVE,380.0,ITALIA,RECUPERADO,<built-in function all>,<built-in function all>,<built-in function all>,13/3/2020 0:00:00,PCR,6.0,<built-in function all>
1,<built-in function all>,2,<built-in function all>,76,VALLE,76111,BUGA,34,1,M,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,<built-in function all>,<built-in function all>,<built-in function all>,19/3/2020 0:00:00,PCR,5.0,<built-in function all>
2,<built-in function all>,3,<built-in function all>,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,<built-in function all>,<built-in function all>,<built-in function all>,15/3/2020 0:00:00,PCR,6.0,<built-in function all>
3,<built-in function all>,4,<built-in function all>,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,RELACIONADO,CASA,LEVE,0.0,0,RECUPERADO,<built-in function all>,<built-in function all>,<built-in function all>,26/3/2020 0:00:00,PCR,6.0,<built-in function all>
4,<built-in function all>,5,<built-in function all>,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,RELACIONADO,CASA,LEVE,0.0,0,RECUPERADO,<built-in function all>,<built-in function all>,<built-in function all>,23/3/2020 0:00:00,PCR,6.0,<built-in function all>
5,<built-in function all>,6,<built-in function all>,5,ANTIOQUIA,5360,ITAGUI,27,1,F,RELACIONADO,CASA,LEVE,0.0,0,RECUPERADO,<built-in function all>,<built-in function all>,<built-in function all>,26/3/2020 0:00:00,PCR,6.0,<built-in function all>
6,<built-in function all>,7,<built-in function all>,13001,CARTAGENA,13001,CARTAGENA,85,1,F,IMPORTADO,CASA,LEVE,840.0,ESTADOS UNIDOS DE AMÉRICA,RECUPERADO,<built-in function all>,<built-in function all>,<built-in function all>,17/3/2020 0:00:00,PCR,6.0,<built-in function all>
7,<built-in function all>,8,<built-in function all>,11,BOGOTA,11001,BOGOTA,22,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,<built-in function all>,<built-in function all>,<built-in function all>,21/3/2020 0:00:00,PCR,6.0,<built-in function all>
8,<built-in function all>,9,<built-in function all>,11,BOGOTA,11001,BOGOTA,28,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,<built-in function all>,<built-in function all>,<built-in function all>,23/3/2020 0:00:00,PCR,6.0,<built-in function all>
9,<built-in function all>,10,<built-in function all>,11,BOGOTA,11001,BOGOTA,36,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,<built-in function all>,<built-in function all>,<built-in function all>,21/3/2020 0:00:00,PCR,6.0,<built-in function all>


#Euclidean function



In [ ]:
df_to_use_in_pca = df.iloc[:, lambda df: [3,5,7,9,10,11,12,13,15,16,17,18,20,21]].copy() 
df_to_use_in_pca.info()

In [ ]:
#Convert the DataFrame to use best possible dtypes.
dfn = df_to_use_in_pca.convert_dtypes()
dfn

In [ ]:
dfn.info()

#Compromiso

1. Servicio que colecta la data por departamento y la guarda en la base de datos
    * Eliminar registros por departamento y volver a consultarlos y almacenarlos
2. Servicio que recibe por parametro el id del departamento 
3. Servicio que recupera la info por municipio.


#Commitment

1. Service which collect the data from Colombia according to each departament (state)
    * Delete records per departament and request it again
2. Service which get by parameter the departament_id and return the information
3. Service get municipality info


Documentation



request
[location and count](https://datos.gov.co/resource/gt2j-8ykr.json?$select=%20count(id_de_caso),ubicacion&$group=ubicacion)

Json
>```
count_id_de_caso	"11015"
ubicacion	"casa"
count_id_de_caso	"4028443"
ubicacion	"Casa"
count_id_de_caso	"7"
ubicacion	"CASA"
count_id_de_caso	"105326"
ubicacion	"Fallecido"
count_id_de_caso	"22865"
ubicacion	"Hospital"
count_id_de_caso	"4963"
ubicacion	"Hospital UCI"
count_id_de_caso	"14575"
ubicacion	"N/A"
```



Contagion types


**source contagion type**
[request](https://datos.gov.co/resource/gt2j-8ykr.json?$select=%20fuente_tipo_contagio&$group=fuente_tipo_contagio)

Json
>```
[
{"fuente_tipo_contagio":"Comunitaria"}
,{"fuente_tipo_contagio":"En estudio"}
,{"fuente_tipo_contagio":"Importado"}
,{"fuente_tipo_contagio":"Relacionado"}
]
```


Patient State [request](https://datos.gov.co/resource/gt2j-8ykr.json?$select=%20estado&$group=estado)


***Patient state *** 

>```
[{"estado":"Fallecido"}
,{"estado":"Grave"}
,{"estado":"leve"}
,{"estado":"Leve"}
,{"estado":"LEVE"}
,{"estado":"Moderado"}
,{"estado":"N/A"}]
```



#PCA

## Example of PCA

In [ ]:
from sklearn.decomposition import PCA

#Example 1
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

In [ ]:
#Example 2

In [ ]:
pca = PCA(n_components=2, svd_solver='full')
pca.fit(X)

In [ ]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

#Bayesian Algoritm

In [ ]:
#Clustering


Advising by Ian Tomas, after meeting with Michael M. and Sebastian Romero L.

>A good place to start would be a similarity measure in your data, which you can implement with a radial basis function. This function inputs two records from your data set and outputs a single number between 0 and 1 which represents how similar the two records are, where a value closer to 1 means they are more similar, closer to 0 means they are different. This is a good idea to use if you want to represent your data as a graph as typically the similarity measure can work as an edge weight in a graph.

```
 void vectorsubtraction(vector<float> v1, vector<float> v2, vector<float> &v3){

    int i;
    //vectors must be same size
    if(v1.size() != v2.size()) return;
    for(i = 0; i < v1.size(); i++){
        //if symbolic attribute, push 1 to represent not equal, 0 to represent equal
        if(symbolic(i)){
          v3.push_back(v1[i] != v2[i]);
          //if continous, subtract values
        } else {
          v3.push_back((v1[i] - v2[i]));
        }
    }
}

float normalizevector(vector<float> v){

    int i;
    float total = 0.0;
    for(i = 0; i < v.size(); i++){
        total += (v[i] * v[i]);
    }

    return sqrt(total);
}

double edgeweight(vector<float> v1, vector<float> v2){

    vector<float> v3;
    vectorsubtraction(v1, v2, v3);

    float normalized = normalizevector(v3);
    double base = exp(normalized);
    return (1 / base);
}
```



In [ ]:
#Python code of barycentric clustering


In [ ]:
#from sklearn.preprocessing import OneHotEncoder

In [ ]:
df_np = df.to_numpy()
print(df_np)
df.info()

In [ ]:
df_to_use_in_pca = df_to_use_in_pca.apply(pd.to_numeric)

In [ ]:
for i in df_to_use_in_pca:
  df_to_use_in_pca[i].astype(int)

enc = OneHotEncoder(handle_unknown='ignore')
converted = df_to_use_in_pca.to_numpy()
print(converted)
enc.fit(converted)

#Label Encoding

In [ ]:
#from sklearn.preprocessing import LabelEncoder

In [ ]:
df.columns

In [ ]:
df['case_location_encoded'] = LabelEncoder().fit_transform(df['case_location'])
df[['case_location', 'case_location_encoded']] # special syntax to get just these two columns

In [ ]:
df.head(200)

In [ ]:
print(df.dtypes)

Statistics # Statistics to develop